# Process JHU Data

This notebook downloads the latest JHU data from GitHub, processes it for dashboard visualization, and places it in the published folder.

It uses output generated by the `generate_points_from_JHU` notebook.

### Papermill

In [ ]:
# parameters
data_dir = '/opt/src/data'

For papermill execution, the pameters are:
- data_dir: That data directory to read data from and publish data to.

### Google API authentication

In [ ]:
import os
google_refresh_token = os.environ.get('CHM_GOOGLEAPI_REFRESH_TOKEN', '')
google_client_id = os.environ.get('CHM_GOOGLEAPI_CLIENT_ID', '')
google_client_secret = os.environ.get('CHM_GOOGLEAPI_CLIENT_SECRET', '')

Get the refresh token and client information from the environment. This needs to be set in the environment for local development, and is pulled from GitHub secrets in the data processing GitHub actions.

In [ ]:
if not google_refresh_token:
    print('NOTE: No google authentication information found; EAC data will not be processed.')

This configures the spreadsheet information for reading the specific sheet filled out by the EAC:

In [ ]:
EAC_SHEET_ID ='1PVF0mrP9bXgCke-snlHaxaR0Z5CIXwRRFL99HSazDR0'
EAC_SHEET_DATA_RANGE = 'Data!A1:U'

# Make True to use test rows locally.
EAC_USE_TEST_ROWS = False

In [ ]:
import json
import io
import os
import re
from datetime import datetime
from collections import defaultdict

import requests
import numpy as np
import pandas as pd
from shapely.geometry import Point, shape
from slugify import slugify

from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

In [ ]:
eac_countries = [
    'Burundi',
    'Kenya',
    'Rwanda',
    'South Sudan',
    'Tanzania',
    'Uganda'
]

In [ ]:
def get_code(admin0, admin1=None, admin2=None):
    """Generates a code from JHU data names, which is used to connect
    data to feature IDs."""
    slug_txt = admin0
    if admin1 is not None:
        slug_txt = "{} {}".format(admin1, slug_txt)
    if admin2 is not None:
        slug_txt = "{} {}".format(admin2, slug_txt)
    return slugify(slug_txt)

def fetch_df(url):
    """Fetches a Pandas DataFrame from a remote source"""
    r = requests.get(url)
    return pd.read_csv(io.BytesIO(r.content))
    

In [ ]:
with open(os.path.join(data_dir, 'case-codes-to-ids-intermidiate.json')) as f:
    case_codes_to_ids = json.loads(f.read())
with open(os.path.join(data_dir, 'case-codes-to-alpha2.json')) as f:
    case_codes_to_alpha2 = json.loads(f.read())

Fetch the JHU data from it's source on GitHub.

In [ ]:
cases_df = fetch_df('https://github.com/CSSEGISandData/COVID-19/raw/master/'
                    'csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deaths_df = fetch_df('https://github.com/CSSEGISandData/COVID-19/raw/master/'
                     'csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
recovered_df= fetch_df('https://github.com/CSSEGISandData/COVID-19/raw/master/'
                       'csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

US county data. This data only has cases and deaths.

In [ ]:
us_cases_df = fetch_df('https://github.com/CSSEGISandData/COVID-19/raw/master/'
                    'csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
us_deaths_df = fetch_df('https://github.com/CSSEGISandData/COVID-19/raw/master/'
                     'csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')

In [ ]:
us_territories = [
    'American Samoa',
    'Guam',
    'Northern Mariana Islands',
    'Puerto Rico',
    'Virgin Islands'
]

def filter_us(df):
   # Filter out counties that have 0 latest data.
    filtered_df = df[df.iloc[:,-1] != 0]
    filtered_df = filtered_df[
        (filtered_df['Province_State'].isin(us_territories)) |
        (
            (~filtered_df['Lat'].isnull()) &
            (filtered_df['Lat'] != 0.0) &
            (~filtered_df['FIPS'].isnull())
        )
    ]
    
    return filtered_df

def reformat_us(df):
    columns_to_drop = [
        'UID',
        'iso2',
        'iso3',
        'code3',
        'FIPS',
        'Admin2',
        'Province_State',
        'Country_Region',
        'Combined_Key',
        'Long_'
    ]
    
    def set_prov(row):        
        prov = row['Province_State']
        if row['Admin2'] and (type(row['Admin2']) != float or not np.isnan(row['Admin2'])):
            prov = '{} {}'.format(row['Admin2'], prov)
        return prov

    formatted_df = df.copy()
    formatted_df['Province/State'] = formatted_df.apply(set_prov, axis=1)
    formatted_df['Country/Region'] = formatted_df['Country_Region']
    formatted_df['Long'] = formatted_df['Long_']
    formatted_df = formatted_df.drop(columns=columns_to_drop)
    return formatted_df
    
formatted_us_cases_df = reformat_us(filter_us(us_cases_df))
formatted_us_deaths_df = reformat_us(filter_us(us_deaths_df))

Map out the formatted dates for each date column.

In [ ]:
non_date_columns = ['Province/State', 'Country/Region', 'Lat', 'Long']
date_columns = list(set(cases_df.columns) - set(non_date_columns))

dates_to_format = {}
for d in date_columns:
    dt = datetime.strptime(d, '%m/%d/%y')
    dates_to_format[d] = dt.strftime('%Y-%m-%d')

#### Gather JHU data organized by region and date

This code gathers the data in an intermediate dictionary, keyed to a region ID that made out of a tuple of the admin0 and admin1 column values. The values represent the total confirmed cases, deaths, and recovered patients for each date.

In [ ]:
jhu_case_data = {}
jhu_county_case_data = {}
alpha2_to_id = {}

new_key = max(case_codes_to_ids.values()) + 1

def add_new_region(code, body):
    global new_key
    this_key = new_key
    jhu_case_data[new_key] = body
    case_codes_to_ids[code] = new_key
    new_key += 1 
    alpha2_to_id[body['a2']] = this_key
    return this_key

# Setup multi-national entries
# Data points with the 'points' propertyset to None don't display on the map.
global_key = add_new_region('global', { 'a2': 'XG', 'map': False, 'dates': {} })
eac_key = add_new_region('eac', { 'a2': 'XE', 'map': False, 'dates': {} })

# Setup nations with only sub-regions to be totaled up
# so that we can display figures at a national level.
nations_to_total = {
    'Australia': None, 
    'Canada': None, 
    'China': None
}
for nation in nations_to_total:
    nations_to_total[nation] = add_new_region(get_code(nation), { 
        'a2': case_codes_to_alpha2[get_code(nation)],
        'map': False,
        'dates': {}
    })

def add_multiregion_cases(key, dt, cases):
    if not dt in jhu_case_data[key]['dates']:
        jhu_case_data[key]['dates'][dt] = [cases, 0, 0]
    else:
        (prev_c, _, _) = jhu_case_data[key]['dates'][dt]
        jhu_case_data[key]['dates'][dt] = [cases + prev_c, 0, 0]
        
def add_multiregion_deaths(key, dt, deaths):
    (prev_c, total_d, _) = jhu_case_data[key]['dates'][dt]
    if total_d is None:
        total_d = deaths
    else:
        total_d += deaths
    jhu_case_data[key]['dates'][dt] = [prev_c, total_d, 0]
    
def add_multiregion_recovered(key, dt, recovered):
    (prev_c, prev_d, total_r) = jhu_case_data[key]['dates'][dt]
    if total_r is None:
        total_r = recovered
    else:
        total_r += recovered
    jhu_case_data[key]['dates'][dt] = [prev_c, prev_d, total_r]
    
def should_skip(row):
    # Skip the cruise ships
    lat = row['Lat']
    lng = row['Long']
    if (lat, lng) == (0, 0):
        print('Skipping Cruise Ship {}, {}'.format(row['Province/State'], 
                                                   row['Country/Region']))
        return True
    
    # Skip unassigned and 'out-of' counties
    if type(row['Province/State']) is not float and (
        row['Province/State'].startswith('Out of') or
        row['Province/State'].startswith('Unassigned')):
        print('Skipping odd county data {}, {}'.format(row['Province/State'], 
                                                       row['Country/Region']))
        return True

    return False

def should_map(code):
    #return code != 'us'
    return True

def get_region_info(row):
    admin0 = row['Country/Region']
    admin1 = row['Province/State']
    if type(admin1) is float and np.isnan(admin1):
        admin1 = None

    code = get_code(admin0, admin1)
    country_a2 = case_codes_to_alpha2[code]
    region_id = case_codes_to_ids[code]
    
    is_country = admin1 is None
    
    return (code, country_a2, region_id, is_country)

def process_cases(row, is_county):
    if should_skip(row):
        return
    
    data = jhu_county_case_data if is_county else jhu_case_data
        
    admin0 = row['Country/Region']
    code, country_a2, region_id, is_country = get_region_info(row)

    data[region_id] = { 
        'a2': country_a2,
        'map': should_map(code),
        'dates': {} 
    }
    
    if(is_country):
        alpha2_to_id[country_a2] = region_id
    
    for d in date_columns:
        cases = row[d]        
        dt = dates_to_format[d]
        data[region_id]['dates'][dt] = [cases, 0, 0]
        
        if not is_county:
            jhu_case_data[region_id]['dates'][dt] = [cases, 0, 0]
            add_multiregion_cases(global_key, dt, cases)

            if admin0 in eac_countries:
                add_multiregion_cases(eac_key, dt, cases)

            if admin0 in nations_to_total:
                add_multiregion_cases(nations_to_total[admin0], dt, cases)
            
def process_deaths(row, is_county):
    if should_skip(row):
        return
    
    data = jhu_county_case_data if is_county else jhu_case_data
        
    admin0 = row['Country/Region']
    code, country_a2, region_id, _ = get_region_info(row)

    for d in date_columns:
        deaths = row[d]
        dt = dates_to_format[d]

        if region_id not in data:
            print(' - Region "{}" in deaths but not in cases'.format(code))
            continue
            
        if dt not in data[region_id]['dates']:
            print(' - Date {} in deaths but not in cases'.format(dt))
            continue
            
        data[region_id]['dates'][dt][1] = deaths
        
        if not is_county:
            add_multiregion_deaths(global_key, dt, deaths)

            if admin0 in eac_countries:
                add_multiregion_deaths(eac_key, dt, deaths)

            if admin0 in nations_to_total:
                add_multiregion_deaths(nations_to_total[admin0], dt, deaths)
            
def process_recovered(row, is_county):
    if should_skip(row):
        return
    
    data = jhu_county_case_data if is_county else jhu_case_data
        
    admin0 = row['Country/Region']
    code, country_a2, region_id, _ = get_region_info(row)

    for d in date_columns:
        recovered = row[d]
        dt = dates_to_format[d]
        
        if region_id not in data:
            print(' - Region "{}" in recovered but not in cases'.format(code))
            continue
            
        if dt not in data[region_id]['dates']:
            print(' - Date {} in recovered but not in cases'.format(dt))
            continue
        
        # Skip canada as it doesn't match up with the other datasets
        if code != 'canada':
            data[region_id]['dates'][dt][2] = recovered

        if not is_county:
            add_multiregion_recovered(global_key, dt, recovered)

            if admin0 in eac_countries:
                add_multiregion_recovered(eac_key, dt, recovered)

            if admin0 in nations_to_total:
                add_multiregion_recovered(nations_to_total[admin0], dt, recovered)

for _, row in cases_df.iterrows():
    process_cases(row, is_county=False)
    
for _, row in formatted_us_cases_df.iterrows():
    process_cases(row, is_county=True)

for _, row in deaths_df.iterrows():
    process_deaths(row, is_county=False)
    
for _, row in formatted_us_deaths_df.iterrows():
    process_deaths(row, is_county=True)
    
for _, row in recovered_df.iterrows():
    process_recovered(row, is_county=False)

In [ ]:
for x in nations_to_total:
    assert nations_to_total[x] in jhu_case_data

The max date and latest counts for EAC countries:

In [ ]:
max_date = sorted(jhu_case_data[eac_key]['dates'], reverse=True)[0]
print('MAX DATE: {}'.format(max_date))
print('\nCountry counts:')
for x in eac_countries:
    print('  {}: {}'.format(x, jhu_case_data[case_codes_to_ids[
        get_code(x)]]['dates'][max_date]))


#### Layer in data from EAC

Grab the spreadsheet that's fed from the form being filled out by EAC.

Notice that if there ae multiple entries for a date, the last row with that date will take effect. This means that if EAC notices a mistake, they can resubmit for that day and the new data will be used.

In [ ]:
def update_rollup(rollup_id, date, cases, deaths, recovered):
    jhu_numbers = jhu_case_data[rollup_id]['dates'].get(date)
    if jhu_numbers is not None:
        jhu_case_data[rollup_id]['dates'][date] = [
            jhu_numbers[0] + cases,
            jhu_numbers[1] + deaths,
            jhu_numbers[2] + recovered
        ]

def update_eac_numbers(region_id, date, cases, deaths, recovered):
    # Update the specific country and any roll-ups.
    # If a roll-up doesn't have that date, don't add it.
    jhu_numbers = jhu_case_data[region_id]['dates'].get(date)
    
    if jhu_numbers is not None:
        for rollup_id in [eac_key, global_key]:
            update_rollup(
                rollup_id,
                date,
                -jhu_numbers[0],
                -jhu_numbers[1],
                -jhu_numbers[2]
            )
    
    jhu_case_data[region_id]['dates'][date] = [
        eac_numbers['cases'],
        eac_numbers['deaths'],
        eac_numbers['recovered']
    ]
    
    if jhu_numbers is not None:
        for rollup_id in [eac_key, global_key]:
            update_rollup(
                rollup_id,
                date,
                cases,
                deaths,
                recovered
            )

In [ ]:
def get_spreadsheet_data():
    creds = Credentials(token=None,
                        refresh_token=google_refresh_token,
                        token_uri='https://oauth2.googleapis.com/token',
                        client_id=google_client_id,
                        client_secret=google_client_secret)

    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets()
    response = sheet.values().get(
        spreadsheetId=EAC_SHEET_ID, 
        range=EAC_SHEET_DATA_RANGE
    ).execute()
    result = response.get('values', None)
    if result is None:
        return None
    return pd.DataFrame(result[1:], columns=result[0])

In [ ]:
eac_sheet_df = get_spreadsheet_data()

In [ ]:
eac_val_col_mapping = {
    'Confirmed Cases': 'cases',
    'Recovered Cases': 'recovered',
    'Deaths': 'deaths'
}

eac_country_to_id = {
    'Burundi': alpha2_to_id['BI'],
    'Kenya': alpha2_to_id['KE'],
    'Rwanda': alpha2_to_id['RW'],
    'South Sudan': alpha2_to_id['SS'],
    'Tanzania': alpha2_to_id['TZ'],
    'Uganda': alpha2_to_id['UG']
}

eac_case_data = {}
for region_id in eac_country_to_id.values():
    eac_case_data[region_id] = defaultdict(dict)

for _, row in eac_sheet_df.iterrows():
    dt = datetime.strptime(row['Timestamp of the data'], '%m/%d/%Y')
    date = dt.strftime('%Y-%m-%d')
    is_test = row['IS_TEST'] == 'Y'
    
    
    for col in eac_sheet_df.columns:
        m = re.match('(Confirmed Cases|Recovered Cases|Deaths) \((.*)\)', 
                     col)
        if m:
            value_type = eac_val_col_mapping[m.group(1)]
            country = m.group(2)
            region_id = eac_country_to_id[country]
            
            if not is_test or EAC_USE_TEST_ROWS:
                eac_case_data[region_id][date][value_type] = int(row[col])


Override or add to JHU data.

In [ ]:
for country_name, region_id in eac_country_to_id.items():
    jhu_country_info = jhu_case_data[region_id]
    
    for date in eac_case_data[region_id]:
        eac_numbers = eac_case_data[region_id][date]
        update_eac_numbers(
            region_id, 
            date,
            eac_numbers['cases'],
            eac_numbers['deaths'],
            eac_numbers['recovered']
        )
        print('USING EAC NUMBERS FOR {} ON {}'.format(country_name, date))

#### Consolidate data

Go through each regions case data, compute active and daily changes, and construct final data json objects.

In [ ]:
def consolidate_data(data):
    result = {}
    for region_id, region_data in data.items():
        result[region_id] = {}
        result[region_id]['a2'] = region_data['a2']
        result[region_id]['map'] = region_data['map']

        result[region_id]['dates'] = {}

        (prev_cases,
         prev_deaths,
         prev_recovered,
         prev_active) = (None,
                         None,
                         None,
                         None)

        for date in sorted(region_data['dates']):

            (cases,
             deaths,
             recovered) = region_data['dates'][date]

            active = cases - deaths - recovered

            (cases_change,
             deaths_change,
             recovered_change,
             active_change) = (None,
                               None,
                               None,
                               None)

            if prev_cases is not None:
                cases_change = cases - prev_cases
                deaths_change = deaths - prev_deaths
                recovered_change = recovered - prev_recovered
                active_change = active - prev_active


            result[region_id]['dates'][date] = {
                'cases': cases,
                'cases_change': cases_change,
                'deaths': deaths,
                'deaths_change': deaths_change,
                'recovered': recovered,
                'recovered_change': recovered_change,
                'active': active,
                'active_change': active_change,
            }

            (prev_cases,
             prev_deaths,
             prev_recovered,
             prev_active) = (cases,
                             deaths,
                             recovered,
                             active)
    return result

In [ ]:
case_data = consolidate_data(jhu_case_data)
county_case_data = consolidate_data(jhu_county_case_data)

In [ ]:
violations = 0
for code, region_id in case_codes_to_ids.items():
    if region_id not in case_data and region_id not in county_case_data:
        violations += 1
        print('Region {} does not have data!'.format(code))
if violations > 0:
    raise Exception('{} regions do not have data!'.format(violations))

Get data used in the configuration.

In [ ]:
ids_to_max_dates = {}
all_dates = set([])
def max_time(region_id, region_data):
    return sorted(map(lambda x: (x, datetime.strptime(x, '%Y-%m-%d')), 
                  region_data['dates'].keys()), key=lambda x: x[1])[-1][0]
   
for region_id in case_codes_to_ids.values():
    if region_id in case_data:
        ids_to_max_dates[region_id] = max_time(region_id, case_data[region_id])
        for d in case_data[region_id]['dates'].keys():
            all_dates.add(d)
    else:  
        ids_to_max_dates[region_id] = max_time(region_id, county_case_data[region_id])
        for d in county_case_data[region_id]['dates'].keys():
            all_dates.add(d)

In [ ]:
global_id = alpha2_to_id['XG']
case_data[global_id]['dates'][ids_to_max_dates[global_id]]

In [ ]:
with open(os.path.join(data_dir, 'case-country-config.json')) as f:
    country_list = json.loads(f.read())
    
    
config = {
    'maxDates': ids_to_max_dates,
    'countries': country_list,
    'dates': sorted(list(all_dates)),
    'alpha2ToId': alpha2_to_id
}

In [ ]:
with open(os.path.join(data_dir, 'published/case-data.json'), 'w') as f:
    f.write(json.dumps(case_data, sort_keys=True))

with open(os.path.join(data_dir, 'published/county-case-data.json'), 'w') as f:
    f.write(json.dumps(county_case_data, sort_keys=True))

In [ ]:
with open(os.path.join(data_dir, 'published/case-codes-to-ids.json'), 'w') as f:
    f.write(json.dumps(case_codes_to_ids, sort_keys=True))

In [ ]:
with open(os.path.join(data_dir, 'published/case-config.json'), 'w') as f:
    f.write(json.dumps(config, sort_keys=True))